# Libraries

In [1]:
import numpy as np
import datetime as d
import math
import queue
import time
from random import randint
import matplotlib.pyplot as plt

# String to test

In [2]:
str_A = "HHHHELLO"#"ALTRUISTIC" #"ACCOMMODATION"#"ALTRUISTIC"#
str_B = "HELLOL"#"ALGORITHM"#"ANTHROPOMORPHIZATION" #"ALGORITHM"#

# Dynamic Programming

In [3]:
def edit_distance(A, B):
    n = len(A)
    m = len(B)
    ED = np.zeros((n+1,m+1), dtype='int32')
    ptr = np.zeros((n+1,m+1), dtype='int32')
           
    
    for i in range(n+1): 
        ED[i,0] = i
        if i>0: 
            ptr[i,0] = 4 #up
    for j in range(m+1):
        ED[0,j] = j
        if j>0:
            ptr[0,j] = 2 #left
            
    if(n==0 or m==0):
        A2, B2 = alignment(A,B,ptr)
        return ED[n,m], A2, B2
    
    for i in range(1, n+1 ):
        for j in range(1, m+1):
            #MATRIX ED
            diff = 0 if A[i-1] == B[j-1] else 1
            ED[i,j] = min(ED[i-1,j] + 1, ED[i, j-1] + 1, ED[i-1,j-1] + diff)
            
            #TRACE-BACK
            if (ED[i,j] == ED[i-1,j] + 1): #UP : DELETION
                ptr[i,j] = ptr[i,j] | 4 
            if (ED[i,j] == ED[i,j-1] + 1): #lEFT : INSERTION
                ptr[i,j] = ptr[i,j] | 2 
            if (ED[i,j] == ED[i-1,j-1] + diff): #DIAGONAL : SUBSTITUTION
                ptr[i,j] = ptr[i,j] | 1
#     print(ptr)
    A2, B2 = alignment(A,B,ptr)
    return ED[n,m], A2, B2

# Trace & Alignment

In [4]:
def alignment(A, B, ptr, L=None):
        
    # create a trackback line
    if L is None:
        L = []
        row = ptr.shape[0] - 1
        col = ptr.shape[1] - 1
        while (row!=0 or col!=0):
            if (ptr[row,col] == 1)  or  (ptr[row,col] == 7):
                L.append('D')
                row = row - 1
                col = col - 1
            elif (ptr[row,col] == 4) or (ptr[row,col] == 5) or (ptr[row,col] == 6):
                L.append('U')
                row = row - 1
            elif (ptr[row,col] == 2) or (ptr[row,col] == 3):
                L.append('L')
                col = col - 1
            

    #Update A and B with alignment
    A2 = []
    B2 = []
    k = 0
    h = 0
    for i in range(len(L), 0, -1):
        if L[i-1] == 'D':
            A2.append(A[k])
            B2.append(B[h])
            k += 1
            h += 1
        elif L[i-1] == 'U':
            A2.append(A[k])
            B2.append('-')
            k += 1
        elif L[i-1] == 'L':
            A2.append('-')
            B2.append(B[h])
            h += 1

    return ''.join(A2), ''.join(B2) 

# Test dynamic programming method

In [5]:
ED, _A, _B  = edit_distance(str_A, str_B)
print("ED using basic method:", ED, "\nResulting alignment: \n",  _A, "\n", _B)


ED using basic method: 4 
Resulting alignment: 
 HHHHELLO- 
 H---ELLOL


# Upgrade dynamic programming

In [6]:
def linear_space_edit_distance(A,B):
    n = len(A)
    m = len(B)
    ED = np.zeros((m+1), dtype='int32')
    for j in range(m+1):
        ED[j] = j

    for i in range(1, n+1):
        left = i
        dag = i-1
        for j in range(1, m+1):
            diff = 0 if A[i-1] == B[j-1] else 1
            curr = min(ED[j] + 1, left + 1, dag + diff)
            left = curr
            dag = ED[j]
            ED[j] = curr
        
    return ED[-1]

In [7]:
ED = linear_space_edit_distance(str_A, str_B)
print("ED using linear space method:", ED)

ED using linear space method: 4


# Recursive method

In [8]:
def edit_distance_recursive(A, B):
    if len(A)== 0:
        return len(B)
    if len(B) == 0:
        return len(A)
    diff = 0 if A[-1] == B[-1] else 1
       
    return min(edit_distance_recursive(A[:-1], B[:-1]) + diff,
              edit_distance_recursive(A[:-1], B) + 1,
              edit_distance_recursive(A, B[:-1]) +1)

In [9]:
# ED1 = edit_distance_recursive(str_A, str_B)
# print("ED using recursive method:", ED)

# Conquer and Divide

In [25]:
def divide_and_conquer_version1(A,B):
    
    def linear_space_Hirshberg(A,B):
        n = len(A)
        m = len(B)

        # position to divide String A
        h = round(n/2)    
        # init ED 1D array
        ED = np.zeros((m+1), dtype='int32')
        # init Hirshberg 1D array
        H = np.zeros((m+1), dtype='int32')


        for j in range(m+1):
            ED[j] = j


        for i in range(1, n+1):
            #set up the left and Diagonal value of ED on each new row
            left = i
            dag = i-1
            #Left and Diagonal value of H on each new row
            H_left = 0
            H_dag = 0
            for j in range(1, m+1):
                #Compute ED
                diff = 0 if A[i-1] == B[j-1] else 1
                curr = min(ED[j] + 1, left + 1, dag + diff)
                #Compute Array 1D H thanks to result of ED
                if i == h:
                    H[j] = j 
                if (i > h):
                    if (curr == left + 1):
                        H_curr = H_left
                    elif (curr == ED[j] + 1):
                        H_curr = H[j]
                    else:
                        H_curr = H_dag
                    #Update new value of left, dag and curr
                    H_left = H_curr
                    H_dag = H[j]
                    H[j] = H_curr
                #Update ED array     
                left = curr
                dag = ED[j]
                ED[j] = curr

        return np.array([h,H[m]], dtype='int32')
    
    
    def Divide_and_Conquer(A, B, newED=None, newA=None, newB=None):
        n = len(A)
        m = len(B)

        #global lists:
        if newED is None:
            newED = []
        if newA is None:
            newA = []
        if newB is None:
            newB = []

        if (n < 2 or m < 2 ):

            ED, _A, _B = edit_distance(A, B)

            newED.append(ED)
            for word in _A:
                newA.append(word )
            for word in _B:
                newB.append(word)

        else:
            H = linear_space_Hirshberg(A, B)
            print(H[0],H[1])
            print(A[:H[0]], B[:H[1]], A[H[0]:], B[H[1]:] )
            Divide_and_Conquer(A[:H[0]], B[:H[1]], newED, newA, newB)
            Divide_and_Conquer(A[H[0]:], B[H[1]:], newED, newA, newB)

        return np.sum(np.array(newED)), ''.join(newA), ''.join(newB)
    
    ed, _A, _B = Divide_and_Conquer(A,B)
    return ed, _A, _B

In [26]:
ED, _A, _B = divide_and_conquer_version1(str_A, str_B)
print("ED using divide and conquer version 1 = ",ED,"\nResulting Alignment: \n", _A, "\n", _B)

4 1
HHHH H ELLO ELLOL
2 2
EL EL LO LOL
1 1
E E L L
1 1
L L O OL
ED using divide and conquer version 1 =  4 
Resulting Alignment: 
 HHHHELLO- 
 H---ELLOL


In [12]:
def divide_conquer_version2(A, B):
    
    def edit_distance_forward(A, B):
        n = len(A)
        m = len(B) 
        ED = np.zeros((m+1), dtype='int32')

        for j in range(m+1):
            ED[j] = j

        for i in range(1, n+1 ):
            left = i
            dag = i-1
            ED[0] = i
            for j in range(1, m+1):
                #MATRIX ED
                diff = 0 if A[i-1] == B[j-1] else 1
                curr = min(ED[j] + 1, left + 1, dag + diff)
                left = curr
                dag = ED[j]
                ED[j] = curr

        return ED

    def edit_distance_backward(A, B):
        n = len(A)
        m = len(B) 

        ED = np.zeros((m+1), dtype='int32')

        for j in range(m, -1, -1):
            ED[j] = m - j

        for i in range(n-1, -1 , -1):
            left = n-i
            dag = n-i-1
            ED[m] = n-i
            for j in range(m-1, -1, -1):
                #MATRIX ED
                diff = 0 if A[i] == B[j] else 1
                curr = min(ED[j] + 1, left + 1, dag + diff)
                left = curr
                dag = ED[j]
                ED[j] = curr

        return ED

    def find_min_point(row):
        
        #find the minimal value of the row
        min_value = min(row)
        #find index of all minimums
        min_index = np.where(row == min_value)[0]
        
        return min_index[0]

    def Hirshberg(A,B):
        n = len(A)
        m = len(B)

        # position to divide String A
        h = int(n/2) 

        ED_forward = edit_distance_forward(A[:h], B)
        ED_backward = edit_distance_backward(A[h:], B)

        # Adding corresponding elements of these two rows h
        row_h = ED_forward[:] + ED_backward[:]

        # position to divide String B
        h2 = find_min_point(row_h) 

        return np.array([h, h2], dtype='int32')

    def Divide_and_Conquer(A, B, newED=None, newA=None, newB=None):
        n = len(A)
        m = len(B)

        #global lists:
        if newED is None:
            newED = []
        if newA is None:
            newA = []
        if newB is None:
            newB = []

        if (n < 2 or m < 2 ):

            ED, _A, _B = edit_distance(A, B)

            newED.append(ED)
            for word in _A:
                newA.append(word )
            for word in _B:
                newB.append(word)

        else:
            H = Hirshberg(A, B)
            Divide_and_Conquer(A[:H[0]], B[:H[1]], newED, newA, newB)
            Divide_and_Conquer(A[H[0]:], B[H[1]:], newED, newA, newB)

        return np.sum(np.array(newED)), ''.join(newA), ''.join(newB)
    
    ed, _A, _B = Divide_and_Conquer(A,B)
    return ed, _A, _B

In [13]:
ED, _A, _B = divide_conquer_version2(str_A, str_B)
print("ED using divide and conquer version 2 = ", ED,"\nResulting Alignment: \n", _A, "\n", _B)

ED using divide and conquer version 2 =  4 
Resulting Alignment: 
 HHHHELLO- 
 H---ELLOL


# Stripe K

In [16]:
def stripe_edit_distance(A, B):
    n = len(A)
    m = len(B)
    
    ED = np.empty((n+1,m+1))
    ptr = np.zeros((n+1,m+1), dtype='int32')
     
       
    if(n==0 or m==0):
        for i in range(n+1): 
            ED[i,0] = i
            if i>0: 
                ptr[i,0] = 4 #up
        for j in range(m+1):
            ED[0,j] = j
            if j>0:
                ptr[0,j] = 2 #left
    
    else:
        
        ED[:] = math.inf #infinity
        
        # auto set up the threshold K
        k = 1    
        while(abs(n-m) > k):
            k += 1
#         print("k = ",k )

        for j in range(m+1):
            if j < k+1:
                ED[0,j] = j
                ptr[0,j] = 2 if j>0 else 0 #left

        for i in range(n+1):
            if i < k+1:
                ED[i,0] = i
                ptr[i,0] = 4 if i>0 else 0 #Up

        for i in range(1, n+1):
            # set up the threshold window
            a = max(1, i-k)
            b = min(m+1, i+k+1)

            for j in range(a, b):
                #MATRIX ED
                diff = 0 if A[i-1] == B[j-1] else 1
                ED[i,j] = min(ED[i-1,j] + 1, ED[i, j-1] + 1, ED[i-1,j-1] + diff)

             #TRACE-BACK
                if (ED[i,j] == ED[i-1,j] + 1): #UP : DELETION
                    ptr[i,j] = ptr[i,j] | 4 
                if (ED[i,j] == ED[i,j-1] + 1): #lEFT : INSERTION
                    ptr[i,j] = ptr[i,j] | 2 
                if (ED[i,j] == ED[i-1,j-1] + diff): #DIAGONAL : SUBSTITUTION
                    ptr[i,j] = ptr[i,j] | 1
           
    A2, B2 = alignment(A,B,ptr)            
    return ED[n,m], A2, B2

In [17]:
ED, _A, _B  = stripe_edit_distance(str_A, str_B)
print("ED using stripe K method:", ED, "\nResulting alignment: \n",  _A, "\n", _B)

ED using stripe K method: 6.0 
Resulting alignment: 
 HHHHELLO 
 HELLOL--


# Branch and Bound

In [18]:
class Node:
    def __init__(self, A, B, cost):
        self.A = A
        self.B = B
        self.cost = cost
        self.h = heuristic(self.A, self.B) 
        self.f = self.cost + self.h 
        self.path = ''
        
def heuristic(A, B):
    return (abs(len(A) - len(B)))

def Branch_and_Bound_version2(A,B):
    
    Q = queue.LifoQueue()
    node0 = Node(A,B,0)
    bound = math.inf    
    Q.put(node0)
    path = ''
    
    while(Q.empty()==False):
        node = Q.get()
        
        #if the node can be expanded:
        if(len(node.A) != 0 and len(node.B)!= 0):
            #expand to three new nodes
            diff = 0 if node.A[-1] == node.B[-1] else 1
            nextNodes = []
            nextNodes.append(Node(node.A[:-1], node.B, node.cost + 1))
            nextNodes.append(Node(node.A, node.B[:-1],  node.cost + 1))
            nextNodes.append(Node(node.A[:-1], node.B[:-1], node.cost + diff))
            #update the path of nodes
            nextNodes[0].path = node.path + 'U'
            nextNodes[1].path = node.path + 'L'
            nextNodes[2].path = node.path + 'D'
            
            #if the value of node higher than the bound, so cutoff, otherwise put it into the queue to expande next time
            for _node in nextNodes:
                if _node.f <= bound:
                    Q.put(_node)
                    
        #if solution is found, the node cannot be expanded:        
        elif(len(node.A) == 0 or len(node.B)== 0):
            #update the bound if higher than the value of the final node (solution), 
            #and also update the path leading the new solution
            if bound >= node.f:
                path = node.path + len(node.B)*'L' + len(node.A)*'U'
                bound = node.f
                               
                
    _A, _B = alignment(A, B, None, path)
    return bound, _A, _B

In [19]:
ED, _A, _B  = Branch_and_Bound_version2(str_A, str_B)
print("ED using B&B method:", ED, "\nAlignment resulting:\n", _A, "\n", _B)

ED using B&B method: 4 
Alignment resulting:
 HHHHELLO- 
 H---ELLOL


# other traceback version 

In [20]:

# Dynamic Programming Method
def nisal_edit_distance(A, B):
    n = len(A)
    m = len(B)
    ED = np.zeros((n + 1, m + 1), dtype='int32')

    for i in range(m+1):
        ED[0,i] = i

    for j in range(n+1):
        ED[j,0] = j

    for k in range (1,n+1):
        for l in range(1,m+1):
            if A[k-1] == B[l-1]:
                ED[k,l] = ED[k-1, l-1]

            else:
                ED[k,l] = min(ED[k-1, l-1], ED[k, l-1], ED[k-1, l]) + 1

    #traceback
    tr = [] #list of actions
    p=n
    q=m
    while p>=0 or q>=0:
        if p>0 and q>0:
            if A[p-1] == B[q-1]:
                tr.append('D') 
                p= p-1
                q = q-1
            else:
                if ED[p-1,q-1] == min(ED[p-1,q-1],ED[p,q-1],ED[p-1,q]):
                    tr.append('D') #SUBSTITUTION
                    p = p - 1
                    q = q - 1
                elif ED[p,q-1] == min(ED[p-1,q-1],ED[p,q-1],ED[p-1,q]):
                    tr.append('L') #INSERTION
                    q=q-1
                else:
                    tr.append('U') #DELETION
                    p=p-1
        elif p==0 and q>0:
            tr.append('L')
            q=q-1
        elif p>0 and q==0:
            tr.append('U')
            p=p-1
        else:
            p=-1
            q=-1
#     tr.reverse
#   Update A and B with alignment
    A2 = []
    B2 = []
    k = 0
    h = 0
    for i in range(len(tr), 0, -1):
        if tr[i-1] == 'D':
            A2.append(A[k])
            B2.append(B[h])
            k += 1
            h += 1
        elif tr[i-1] == 'U':
            A2.append(A[k])
            B2.append('-')
            k += 1
        elif tr[i-1] == 'L':
            A2.append('-')
            B2.append(B[h])
            h += 1
        
    return ED[n,m], ''.join(A2), ''.join(B2)


In [21]:
ED, _A, _B  = nisal_edit_distance(str_A, str_B)
print("ED using Nisal trace back method:", ED, "\nResulting alignment: \n",  _A, "\n", _B)

ED using Nisal trace back method: 4 
Resulting alignment: 
 HHHHELLO- 
 ---HELLOL


# test time complexity

In [22]:
# def randWord(k):
#     return ''.join([ chr(randint(ord('a'), ord('z'))) for _ in range(k) ])

# def randWords(n):
#     return [np.array([randWord(k), randWord(k)]) for k in range(n)]

# def time2words(a, b, key):
#     t0 = time.time()
#     if key == 1:
#         divide_and_conquer_version1(a,b)
#     elif key == 2:
#         divide_conquer_version2(a,b)
#     elif key == 3:
#          edit_distance(a,b)
#     t1 = time.time()
#     return round(t1-t0, 3)

# words=randWords(400)
# x = range(0, len(words), 5)

# def timeoverall(k):
#     y=[]
#     for pair in words:
#         value = time2words(pair[0], pair[1], key = k)
#         y.append(value)
# #     for i in x:
# #         value = time2words(words[i][0], words[i][1], key = k)
# #         y.append(value)
#     return y

In [23]:
# y1 = timeoverall(1)
# y2 = timeoverall(2)
# y3 = timeoverall(3)

In [24]:
# plt.plot( y3, color='green', label='Dynamic Programming')
# plt.plot( y1, color='blue', label='Divide and conquer version 1')
# plt.plot( y2, color='red', label='Divide and conquer version 2')
# plt.xlabel('Size of the words') 
# plt.ylabel('Time (s)')
# plt.legend()
# # plt.show()
# plt.savefig('anh.png', transparent=False, dpi=200)